<a href="https://colab.research.google.com/github/NarciSource/Aivle--MiniProject-3/blob/main/%EC%A0%80%EC%8B%9C%EB%A0%A5%EC%9E%90%EB%A5%BC_%EC%9C%84%ED%95%9C_%ED%99%94%ED%8F%90_%ED%83%90%EC%A7%80_4%EA%B8%B0(%EC%8B%A4%EC%8A%B5)_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동, 데이터 다운로드
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install gdown

### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
import zipfile, gdown,os
url ="https://drive.google.com/file/d/19SPK9LPuVIyM03w_DbNsqqUbE5Lv80xb/view?usp=sharing"
file_name = "money_dataset.zip"
output = "/content/drive/MyDrive/aivle/" + file_name
if not os.path.exists(output):
    gdown.download(url=url, output=output, quiet=False, fuzzy=True)

In [ ]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(output)

In [ ]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [ ]:
# 2. Dataset metadata 입력
won_list = ['1십', '5십', '1백', '5백', '1천', '5천', '1만', '5만']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

### split-folders
> https://pypi.org/project/split-folders/

split-folders는 파일을 훈련셋과 검증셋으로 나눌 때 사용되는 라이브러리이다.  
각 클래스 폴더별로 비율에 따라 층화표집 된다.  
파일들은 셔플되어서 뽑히며 group_prefix를 설정하면 파일을 그룹화할 수 있다.  
사전순으로 파일 이름이 정렬되어 있으며 anotation이 이미지와 같은 파일 이름이면 파일 순서대로 그룹으로 묶이게 되는 것이다.  

```
money_dataset
├─ 10
│  ├─ 10_12_1.jpg
│  ├─ 10_12_1.json
│  └─ ...
├─ 20
│  ├─ 20_18_1.jpg
│  ├─ 20_18_1.json
│  └─ ...
...
```
인풋 데이터셋이 라벨에 맞게 나누어져있기에 바로 적용하였다.  
그럼으로써 라벨별로 층화표집하게 된다.

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('/content/Dataset', output='/content/Dataset', ratio=(0.8, 0.2, 0), group_prefix=2)

Copying files: 5218 files [00:15, 347.30 files/s]


json에 라벨 표시가 되어있으므로 라벨 카테고리 폴더는 평탄화한다.

In [ ]:
from os import listdir, walk
from shutil import move, rmtree

for folder in ['train','val']:
    for root, directories, files in walk(data_path+folder):
        for file in files:
            if '.jpg' in file:
                move(root+'/'+file, data_path+'images/'+folder)
            elif '.json' in file:
                move(root+'/'+file, data_path+'labels/'+folder)

In [ ]:
for folder in listdir(data_path):
    if folder not in ['images','labels']:
        rmtree(data_path+folder)

In [ ]:
print("images/train", len(listdir('/content/Dataset/images/train')))
print("labels/train", len(listdir('/content/Dataset/labels/train')))
print("images/val", len(listdir('/content/Dataset/images/val')))
print("lables/val", len(listdir('/content/Dataset/labels/val')))

images/train 4172
labels/train 4172
images/val 1042
lables/val 1042


### 백그라운드 이미지 추가

배경 이미지를 추가하여 모델에 사물이 없는 상황을 추가로 학습하게 한다.  
이는 배경의 패턴이나 요소를 사물로 오인하지 않게 하여 오탐지를 감소하고 모델을 견고하게 하려는 목적이다.

In [ ]:
from distutils.dir_util import copy_tree
copy_tree('/content/drive/MyDrive/aivle/mp3/background_images', \
          '/content/Dataset/images/train')

print("Added background images, number of", len(listdir('/content/drive/MyDrive/aivle/mp3/background_images/')))

Added background images, number of 30


### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/5로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/5로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [ ]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

yolo5의 라벨링 포멧의 값은 0과 1사이로 정규화되어야한다.

In [ ]:
labels = ['Ten', 'Fifty', 'Hundred', 'Five_Hundred',
          'Thousand', 'Five_Thousand', 'Ten_Thousand', 'Fifty_Thousand']

import os, json
for folder in temp_list:
    for filename in listdir(json_path+folder):
        read_file = json_path+folder+'/'+filename
        if 'json' in filename:

            with open(read_file,'r') as f:
                j = json.load(f)
                shapes = j['shapes'][0]
                label = labels.index(shapes['label'].replace('_front','').replace('_back',''))
                (x1,y1), (x2,y2) = shapes['points']
                h, w = j['imageHeight'], j['imageWidth']

                write_file = json_path+folder+'/'+filename.replace('json','txt')
                yolo_format = ' '.join(map(str,[label, (x1+x2)/2/w, (y1+y2)/2/h, (x2-x1)/w, (y2-y1)/h]))

                with open(write_file,'w') as wf:
                    wf.write(yolo_format)

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [ ]:
with open('/content/Dataset/money.yaml', 'w') as f :
    f.write('train: /content/Dataset/images/train/\n')
    f.write('val: /content/Dataset/images/val/\n')
    f.write('nc: '+str(len(won_list))+'\n')
    f.write(f'names: {won_list}')

## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
!pip install -r yolov5/requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15994, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 15994 (delta 18), reused 18 (delta 12), pack-reused 15967
Receiving objects: 100% (15994/15994), 14.64 MiB | 25.03 MiB/s, done.
Resolving deltas: 100% (10980/10980), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.4/618.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.2 MB/s eta 0:00:00


### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

### 하이퍼파라미터 최적값 튜닝

* 하이퍼파라미터를 자동으로 튜닝하는 학습을 우선 수행하였다.

In [ ]:
!python yolov5/train.py --img 640 --exist-ok \
                    --epochs 20 --batch 16 \
                    --cache \
                    --evolve \
                    --data Dataset/money.yaml \
                    --weight yolov5s.pt \
                    --project /content/drive/MyDrive/aivle/mp3 \
                    --name trained

train: weights=/content/drive/MyDrive/aivle/mp3/trained/weights/best.pt, cfg=, data=Dataset/money.yaml, hyp=/content/drive/MyDrive/aivle/mp3/trained/hyp_evolve.yaml, epochs=20, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=300, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/aivle/mp3, name=trained, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.00957, lrf=0.01, momentum=0.93207, weight_decay=0.00049, warmup_epochs=3.1393, warmup_momentum=0.84174, warmup_bias_lr=0.09842, box=0.0

In [ ]:
hyp_src = '/content/drive/MyDrive/aivle/mp3/trained/hyp_evolve.yaml'

* 직관에 맞춘 하이퍼파라미터 커스터마이징

In [ ]:
from fileinput import input

options = '''
    degrees: 90
    perspective: 0.001
    flipud: 1.0
    fliplr: 1.0
    copy_paste: 1.0
'''
options = [option.strip().split() for option in options.splitlines() if option]

with input(hyp_src, inplace=True) as f:
    for line in f:
        for key, value in options:
            if key in line:
                print(key+' '+value)
                break
        else:
            print(line, end='')


with input(hyp_src) as f:
    print(*f)

# YOLOv5 Hyperparameter Evolution Results
 # Best generation: 2
 # Last generation: 5
 #    metrics/precision,       metrics/recall,      metrics/mAP_0.5, metrics/mAP_0.5:0.95,         val/box_loss,         val/obj_loss,         val/cls_loss
 #              0.73075,              0.97882,              0.77326,              0.62467,             0.016263,            0.0090371,             0.011508
 
 lr0: 0.00957
 lrf: 0.01
 momentum: 0.93207
 weight_decay: 0.00049
 warmup_epochs: 3.1393
 warmup_momentum: 0.84174
 warmup_bias_lr: 0.09842
 box: 0.05225
 cls: 0.52986
 cls_pw: 1.0151
 obj: 0.98735
 obj_pw: 1.0162
 iou_t: 0.2
 anchor_t: 4.1296
 fl_gamma: 1.0
 hsv_h: 0.01612
 hsv_s: 0.68896
 hsv_v: 0.38876
 degrees: 90
 translate: 0.09293
 scale: 0.5
 shear: 0.0
 perspective: 0.001
 flipud: 1.0
 fliplr: 1.0
 mosaic: 0.95124
 mixup: 0.2
 copy_paste: 1.0
 anchors: 3.0455



* yolo5의 cutout을 활성화하기 위해서 dataloaders.py를 수정

In [ ]:
src = '/content/yolov5/utils/dataloaders.py'
imprt = 'from utils.augmentations import (Albumentations, augment_hsv, classify_albumentations, classify_transforms, copy_paste, cutout,'
codes = '''
            labels = cutout(img, labels, p=0.5)
            nl = len(labels)  # update after cutout
'''

from fileinput import input
with input(src, inplace=True) as f:
    for line in f:
        if 'utils.augmentations' in line:
            print(imprt)
        elif '# Cutouts' in line:
            for code in codes:
                print(code, end='')
        else:
            print(line, end='')

### 모델 학습

In [ ]:
!python yolov5/train.py --img 1024 --exist-ok \
                    --epochs 100 --batch 16 \
                    --cache \
                    --hyp /content/drive/MyDrive/aivle/mp3/trained/hyp_evolve.yaml \
                    --data Dataset/money.yaml \
                    --weight /content/drive/MyDrive/aivle/mp3/trained/weights/best.pt \
                    --project /content/drive/MyDrive/aivle/mp3 \
                    --name trained

train: weights=/content/drive/MyDrive/aivle/mp3/trained/weights/best.pt, cfg=, data=Dataset/money.yaml, hyp=/content/drive/MyDrive/aivle/mp3/trained/hyp_evolve.yaml, epochs=100, batch_size=16, imgsz=1024, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=/content/drive/MyDrive/aivle/mp3, name=trained, exist_ok=True, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.00957, lrf=0.01, momentum=0.93207, weight_decay=0.00049, warmup_epochs=3.1393, warmup_momentum=0.84174, warmup_bias_lr=0.09842, box=


## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
!python yolov5/detect.py --line-thickness 5 \
                    --conf 0.10 --iou 0.15 \
                    --weights /content/drive/MyDrive/aivle/mp3/trained/weights/best.pt \
                    --source /content/drive/MyDrive/aivle/mp3/testset \
                    --project /content/drive/MyDrive/aivle/mp3 \
                    --name result

detect: weights=['/content/drive/MyDrive/aivle/mp3/trained/weights/best.pt'], source=/content/drive/MyDrive/aivle/mp3/testset, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.15, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/aivle/mp3, name=result, exist_ok=False, line_thickness=5, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
image 1/46 /content/drive/MyDrive/aivle/mp3/testset/20230925_031146.jpg: 640x480 5 1천s, 2 5만s, 296.0ms
image 2/46 /content/drive/MyDrive/aivle/mp3/testset/20230925_031225.jpg: 640x480 2 1천s, 2 5만s, 281.8ms
image 3/46 /content/drive/MyDrive/aivle/mp3/testset/20230925_031348.jpg

#### 동영상

In [ ]:
!python yolov5/detect.py --line-thickness 2 \
                    --conf 0.15 --iou 0.25 \
                    --weights /content/drive/MyDrive/aivle/mp3/trained/weights/best.pt \
                    --source https://www.youtube.com/watch?v=Ole8O_zeth8 \
                    --project /content/drive/MyDrive/aivle/mp3 \
                    --name result

detect: weights=['/content/drive/MyDrive/aivle/mp3/trained/weights/best.pt'], source=https://www.youtube.com/watch?v=Ole8O_zeth8, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.15, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/aivle/mp3, name=result, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

1/1: https://www.youtube.com/watch?v=Ole8O_zeth8...  Success (7836 frames 1280x720 at 29.97 FPS)

0: 384x640 6 1백s, 3 5백s, 1 1만, 2 5만s, 498.7ms
0: 384x640 6 1백s, 3 5백s, 1 1만, 2 5만s, 392.7ms
0: 384x64

In [ ]:
!python yolov5/detect.py --line-thickness 2 \
                    --conf 0.15 --iou 0.25 \
                    --weights /content/drive/MyDrive/aivle/mp3/trained/weights/best.pt \
                    --source https://www.youtube.com/watch?v=b9y4mkYYHnM \
                    --project /content/drive/MyDrive/aivle/mp3 \
                    --name result

detect: weights=['/content/drive/MyDrive/aivle/mp3/trained/weights/best.pt'], source=https://www.youtube.com/watch?v=b9y4mkYYHnM, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.15, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/aivle/mp3, name=result, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

[ERROR:0@5.884] global cap.cpp:164 open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.8.0) /io/opencv/modules/videoio/src/cap_images.cpp:235: error: (-5:Bad argument) CAP_IMAGES: error, expe

In [ ]:
!python yolov5/detect.py --line-thickness 2 \
                    --conf 0.25 --iou 0.25 \
                    --weights /content/drive/MyDrive/aivle/mp3/trained/weights/best.pt \
                    --source /content/drive/MyDrive/aivle/mp3/test/KakaoTalk_20230925_111943635.mp4 \
                    --project /content/drive/MyDrive/aivle/mp3 \
                    --name result

detect: weights=['/content/drive/MyDrive/aivle/mp3/trained/weights/best.pt'], source=/content/drive/MyDrive/aivle/mp3/test/KakaoTalk_20230925_111943635.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.25, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/aivle/mp3, name=result, exist_ok=False, line_thickness=2, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-224-g6262c7f Python-3.10.12 torch-2.0.1+cu118 CPU

Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs
video 1/1 (1/584) /content/drive/MyDrive/aivle/mp3/test/KakaoTalk_20230925_111943635.mp4: 384x640 1 5만, 276.3ms
video 1/1 (2/584) /content/drive/MyDrive/aivle/mp3/test/KakaoTalk_20230925_111943635.mp4: 384x640 1 5만, 263.0ms
video 1/1 (3/584) /con